In [274]:
import numpy as np
import pandas as pd
import pickle

In [275]:
with open('./data/raw/session_train.pkl', 'rb') as picklefile:
    session = pickle.load(picklefile)

In [276]:
with open('./data/processed/y_train.pkl', 'rb') as picklefile:
    y_train = pickle.load(picklefile)

In [277]:
y_train

,country
id,
ppr8sr6z33,NDF
eiwzl341qa,NDF
lor9srems2,NDF
uk2031aezt,NDF
q7k5aflli0,NDF
eswm7wig2e,NDF
036qtdh8u5,NDF
8692718hr3,NDF
iblg556q1g,NDF


In [278]:
y_train = pd.DataFrame(y_train, columns=['country'])
y_bin = y_train
y_bin['booking'] = 1- (y_bin['country'] == 'NDF').astype(int)

In [279]:
# Check the na items
session.isna().sum()

user_id              0
action            3519
action_type      44395
action_detail    44395
device_type          0
secs_elapsed      5235
dtype: int64

In [280]:
# Replace NaN in action column with others
session['action'].fillna("catch-all", inplace=True)
# Clip the sessions longer than 24 hours to 24 hours
session.loc[session['secs_elapsed']>24*3600, 'secs_elapsed'] = 24*3600

In [281]:
# Impute the missing action_type and action_details with mode values
def impute_mode(df, col_key, col_na, mode_table=None):
    
    if (mode_table == None):
        mode_series = df[~df[col_na].isna()].groupby(col_key)[col_na].agg(lambda x:x.value_counts().index[0])
        mode_series = pd.DataFrame(mode_series.reset_index())
    else:
        mode_series = mode_table
    merged_frame = pd.merge(df, mode_series, how='left', 
                             left_on=[col_key], 
                             right_on=[col_key])
    
    merged_frame.rename(columns={col_na+'_x': col_na, col_na+'_y':'mode_value'}, inplace=True)
    merged_frame[col_na].fillna(merged_frame.mode_value, inplace=True)
    merged_frame.drop(['mode_value'], axis=1, inplace=True)
    
    return merged_frame, mode_table

In [282]:
session, action_type_mode = impute_mode(session, 'action', 'action_type')
session, action_detail_mode = impute_mode(session, 'action', 'action_detail')
session.fillna("catch-all", inplace=True)

In [283]:
session.isna().any()

user_id          False
action           False
action_type      False
action_detail    False
device_type      False
secs_elapsed     False
dtype: bool

In [287]:
session.head(50)

,action,action_type,action_detail,device_type,secs_elapsed
user_id,,,,,
jrqykh9y8x,show,view,p3,Mac Desktop,12008
jrqykh9y8x,show,view,p3,Mac Desktop,10696
jrqykh9y8x,show,view,p3,Mac Desktop,12145
szwzfcny9p,show,view,p3,iPad Tablet,1325
77z0fp7mro,show,view,p3,Windows Desktop,1050
bik2hlivrc,show,view,p3,Mac Desktop,401
1lox83p90z,lookup,catch-all,catch-all,Mac Desktop,1670
k02kmpfqyh,show,view,p3,Mac Desktop,3685
3hy9yi6hi8,show,view,p3,Mac Desktop,3602


In [285]:
# Find out what actions are unique to persons who make booking

In [286]:
# Merge session data with y_bin
session.set_index('user_id', inplace=True)
session_y = pd.merge(session, y_bin, how='left', on=session.index)

KeyError: "Index(['jrqykh9y8x', 'jrqykh9y8x', 'jrqykh9y8x', 'szwzfcny9p', '77z0fp7mro',\n       'bik2hlivrc', '1lox83p90z', 'k02kmpfqyh', '3hy9yi6hi8', 'aeej48jx7x',\n       ...\n       '04y8115avm', '04y8115avm', '04y8115avm', '04y8115avm', '04y8115avm',\n       '04y8115avm', '04y8115avm', '04y8115avm', '04y8115avm', '04y8115avm'],\n      dtype='object', name='user_id', length=391140) not in index"